# Fine Tuning an LLM

## Load and Pre-process the dataset

In [2]:
from datasets import load_dataset

ds = load_dataset("Biddls/Onion_News")
ds['train'][0]

{'text': 'Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage #~# LOS ANGELES—Admitting that she’d made some major lifestyle changes since developing her famous KonMari method, a relaxed Marie Kondo told reporters Tuesday that she was now perfectly happy living in waist-high sewage. “The truth is, while I used to be very hard on myself about keeping everything clean, I’m now able to find peace living my life half-submerged in a large, fetid pool of human waste,” said Kondo, who added that while things like tidiness, organization, and minimalism used to spark joy for her, she now felt that same warmth from wading, floating, and swimming in the many gallons of untreated urine and feces that currently filled her home. “It was difficult, but once I had my children, I began to find it impossible to remove, clean, and sanitize the unending stream of excrement that bubbled up out of my toilets and filled my house to the point of collapse. While I used to hate it, I no

In [3]:
import random

SEP = '#~#'

def get_as_messages(data):
    headline = get_headline(data['text'])
    messages = get_prompt()
    messages.append(format_reply(headline))
    return {'messages': messages}
    
def get_headline(text):
    return text.split(SEP)[0].strip()

def get_prompt():
    return [
        {
            "role": "user", 
            "content": get_prompt_no_topic(),
        }
    ]

def get_prompt_no_topic():
    return random.choice([
        'Write a satirical headline in the style of Onion News.',
        'Generate an satirical news headline.',
        "Give me a headline that sounds like it's from the Onion.",
        'Come up with a funny, fake news headline.',
        'Write a headline that is both absurd and oddly believable.',
        'Please write a headline that would feel at home in the Onion.',
        'Can you write a satirical headline for me?',
    ])

def format_reply(headline):
    return {
        "role": "assistant",
        "content": headline,
    }

ds = ds.map(get_as_messages, remove_columns=['text'])
ds['train'][0]

Map:   0%|          | 0/33880 [00:00<?, ? examples/s]

{'messages': [{'content': 'Write a headline that is both absurd and oddly believable.',
   'role': 'user'},
  {'content': 'Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage',
   'role': 'assistant'}]}

## Load the baseline model

In [4]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

torch.random.manual_seed(0)

model_path = "Qwen/Qwen3-4B-Instruct-2507"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto",
)

new_chat_template = \
    "{% for message in messages %}"\
        "{{'<|im_start|>' + message['role'] + '\\n' + message['content'] + '<|im_end|>' + '\\n'}}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '<|im_start|>assistant\\n' }}"\
    "{% endif %}"
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.chat_template = new_chat_template

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training


lora_rank = 16

peft_config = LoraConfig(
    r=lora_rank,
    lora_alpha=2 * lora_rank,
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    bias="none",
)

prepared_model = prepare_model_for_kbit_training(model)

In [7]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir='./qwen3_lora_finetuned_v1.1',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=500,
    report_to="none",
    optim='paged_adamw_8bit',
    lr_scheduler_type='linear',
    warmup_steps=5,
    seed=42,
)

trainer = SFTTrainer(
    model=prepared_model,
    args=training_args,
    train_dataset=ds['train'],
    peft_config=peft_config,
    processing_class=tokenizer,
)

Tokenizing train dataset:   0%|          | 0/33880 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/33880 [00:00<?, ? examples/s]

In [8]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,4.414100
20,1.882900
30,1.694700
40,1.677900
50,1.679900
60,1.609300
70,1.665900
80,1.630100
90,1.600000
100,1.595300


/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be

TrainOutput(global_step=4235, training_loss=1.5585198906827564, metrics={'train_runtime': 3399.4514, 'train_samples_per_second': 9.966, 'train_steps_per_second': 1.246, 'total_flos': 3.261098790027264e+16, 'train_loss': 1.5585198906827564})